In [1]:
import numpy as np
import pandas as pd

In [2]:
 import os
 for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
         print(os.path.join(dirname, filename))

In [3]:
!pip install pyspark

zsh:1: command not found: pip


In [4]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 26.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tutorial').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/17 17:28:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/17 17:28:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
df_train = spark.read.csv('/Users/payel/Downloads/train.csv',header = True,inferSchema=True)

In [8]:
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [9]:
type(df_train)

pyspark.sql.dataframe.DataFrame

In [10]:
df_test = spark.read.csv('/Users/payel/Downloads/test.csv',header=True,inferSchema=True)

In [11]:
type(df_test)

pyspark.sql.dataframe.DataFrame

In [12]:
df_test.show()

+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|          330911| 7.8292| null|       Q|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|          363272|    7.0| null|       S|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|          240276| 9.6875| null|       Q|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0|          315154| 8.6625| null|       S|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|         3101298|12.2875| null|       S|
|        897|     3|Svensson, Mr. Joh...|  male|14.0|    0|    0|            7538|  9.225| null|       S|
|        898|     3|Connolly, Miss. Kate|femal

In [13]:
df_train.count()

891

In [14]:
df_test.count()

418

In [15]:
891+418

1309

In [17]:
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [18]:
df_test.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [19]:
# As the Survived column is missing in test we will create a dummy column and put a value of -1 in it

In [20]:
from pyspark.sql.functions import lit
df_test = df_test.withColumn('Survived', lit(-1))

In [21]:
df_test.show(3)

+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|Survived|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|      -1|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|      -1|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|      -1|
+-----------+------+--------------------+------+----+-----+-----+------+------+-----+--------+--------+
only showing top 3 rows



In [22]:
df_tests = df_test.select('PassengerId','Survived','Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked')

In [23]:
df_tests.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|
|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|
|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [25]:
df_train.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [28]:
df_concat = df_tests.union(df_train)


In [29]:
df_concat.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        892|      -1|     3|    Kelly, Mr. James|  male|34.5|    0|    0|330911|7.8292| null|       Q|
|        893|      -1|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0|363272|   7.0| null|       S|
|        894|      -1|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0|240276|9.6875| null|       Q|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 3 rows



In [30]:
df_concat.count()

1309

In [35]:
type(df_concat)

pyspark.sql.dataframe.DataFrame

In [31]:
# Change the df_concat to pandas to make work easier

In [32]:
df_new = df_concat.toPandas()

In [34]:
type(df_new)

pandas.core.frame.DataFrame

In [36]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int32  
 1   Survived     1309 non-null   int32  
 2   Pclass       1309 non-null   int32  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int32  
 7   Parch        1309 non-null   int32  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int32(5), object(5)
memory usage: 97.3+ KB


In [37]:
# Sorting on passengers

In [38]:
# Sort out female passengers in firstclass

In [64]:
df_femalefirst = df_concat.where((df_concat.Sex == 'female') & (df_concat.Pclass == 1))

In [65]:
print("{0} females out of {1} passengers are in Class1".format(df_femalefirst.count(), df_concat.count()))

144 females out of 1309 passengers are in Class1


In [66]:
type(df_femalefirst)

pyspark.sql.dataframe.DataFrame

In [67]:
df_femalefirst.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+---------------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|   Fare|          Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+---------------+--------+
|        904|      -1|     1|Snyder, Mrs. John...|female|23.0|    1|    0|      21228|82.2667|            B45|       S|
|        906|      -1|     1|Chaffee, Mrs. Her...|female|47.0|    1|    0|W.E.P. 5734| 61.175|            E31|       S|
|        914|      -1|     1|Flegenheim, Mrs. ...|female|null|    0|    0|   PC 17598|31.6833|           null|       S|
|        916|      -1|     1|Ryerson, Mrs. Art...|female|48.0|    1|    3|   PC 17608|262.375|B57 B59 B63 B66|       C|
|        918|      -1|     1|Ostby, Miss. Hele...|female|22.0|    0|    1|     113509|61.9792|            B36|       C|
+-----------+--------+------+-----------

In [68]:
from pyspark.sql.functions import desc

In [69]:
df_femalefirst1 = df_femalefirst.select(df_femalefirst.Name, df_femalefirst.Age).sort(df_femalefirst.Age.desc())

In [70]:
df_femalefirst1.show()

+--------------------+----+
|                Name| Age|
+--------------------+----+
|Cavendish, Mrs. T...|76.0|
|Crosby, Mrs. Edwa...|64.0|
|Compton, Mrs. Ale...|64.0|
|Andrews, Miss. Ko...|63.0|
|Straus, Mrs. Isid...|63.0|
|Stone, Mrs. Georg...|62.0|
|Warren, Mrs. Fran...|60.0|
|Bucknell, Mrs. Wi...|60.0|
|Fortune, Mrs. Mar...|60.0|
|Brown, Mrs. John ...|59.0|
|Lurette, Miss. Elise|58.0|
|Cardeza, Mrs. Jam...|58.0|
|Graham, Mrs. Will...|58.0|
|Bonnell, Miss. El...|58.0|
|Potter, Mrs. Thom...|56.0|
|White, Mrs. John ...|55.0|
|Lindstrom, Mrs. C...|55.0|
|Cornell, Mrs. Rob...|55.0|
|Rothschild, Mrs. ...|54.0|
|Eustis, Miss. Eli...|54.0|
+--------------------+----+
only showing top 20 rows



In [71]:
display(df_new.describe())

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,-0.058060,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.760044,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,-1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,-1.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [73]:
df_new.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,-1,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,None,Q
1,893,-1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,None,S
2,894,-1,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,None,Q
3,895,-1,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,None,S
4,896,-1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,None,S


In [74]:
df_concat.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [75]:
df_concat.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [76]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int32  
 1   Survived     1309 non-null   int32  
 2   Pclass       1309 non-null   int32  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int32  
 7   Parch        1309 non-null   int32  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(2), int32(5), object(5)
memory usage: 97.3+ KB


In [77]:
df_new.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked        True
dtype: bool

In [78]:
df_new.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,-0.058060,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.760044,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,-1.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,-1.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [79]:
from pyspark.sql import functions as F
df_concat.select(F.mean("Age")).show()

+------------------+
|          avg(Age)|
+------------------+
|29.881137667304014|
+------------------+



In [80]:
df_new["Age"]

0       34.5
1       47.0
2       62.0
3       27.0
4       22.0
        ... 
1304    27.0
1305    19.0
1306     NaN
1307    26.0
1308    32.0
Name: Age, Length: 1309, dtype: float64

In [83]:
from pyspark.sql.functions import col

In [84]:
df_concat.groupBy("Embarked").count().orderBy(col("Embarked").desc()).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  914|
|       Q|  123|
|       C|  270|
|    null|    2|
+--------+-----+



In [85]:
df_concat.groupBy("Sex").count().orderBy(col("Sex").desc()).show()

+------+-----+
|   Sex|count|
+------+-----+
|  male|  843|
|female|  466|
+------+-----+

